In [1]:
from snowflake.snowpark.types import Variant
import snowflake.snowpark.functions as F
# def process_coordinates(session: Session) -> Variant:

    # coordinates = session.table('test_points')

    # a = coordinates.to_df('COORDS', 'START')
    # b = coordinates.to_df('COORDS', 'END')

    # joined = a.join(b, F.call_builtin('ST_DISTANCE', F.col('START'), F.col('END')) != 0)

    


In [3]:

from snowflake.snowpark import Session

connection_parameters = {
    "account": 'bxrudoe-ea31380',
    "user": 'LLIU',
    "password": 'Password1',
    "role": 'ACCOUNTADMIN',
    "warehouse": 'COMPUTE_WH',
    "database": 'PROJECT_SANTA',
    "schema": 'ROUTES'    
}
session = Session.builder.configs(connection_parameters).create()
# session.sproc.register(
#     func=process_coordinates
#     ,name='process_coordinates'
#     ,packages=['snowflake-snowpark-python']
#     ,is_permanent=True
#     ,replace=True
#     ,stage_location='@SPROC_STAGE'
# )

In [89]:
# takes a table of of coordinates with a coords column and generates a path that travels to all points 
# using a "nearest neighbor" approach starting from the North Pole



METHOD = 'NAIVE'
START_COL = 'START'
END_COL = 'END'
DISTANCE_COL = 'DISTANCE_KM'
WKT_FMT_COL = 'WKT'

coordinates = session.table('test_points')
num_coords = coordinates.count()

a = coordinates.rename(F.col('COORDS'), START_COL)
b = coordinates.rename(F.col('COORDS'), END_COL)

joined = a.join(b, F.call_builtin('POINTS_EQUAL', F.col(START_COL), F.col(END_COL)) != True)

available_routes = joined \
    .with_column(DISTANCE_COL, F.call_builtin('ST_DISTANCE', F.col(START_COL), F.col(END_COL))/1000) \
    .withColumn(WKT_FMT_COL, F.call_builtin('ST_ASWKT', F.col(END_COL)))


path = []
total_distance = 0
curr_point = f'POINT({0} {90})'
path.append(curr_point)

for _ in range(num_coords):
    next_destination = available_routes.filter(F.call_builtin('POINTS_EQUAL', F.col(START_COL), F.call_builtin('TO_GEOGRAPHY', curr_point))) \
        .orderBy(F.col(DISTANCE_COL)).limit(1) \
        .collect()
    
    next_point = next_destination[0][WKT_FMT_COL]
    path.append(next_point)
    total_distance += next_destination[0][DISTANCE_COL]
    curr_point = next_point
    
    # filter points out from 
    remaining_routes = available_routes.filter(F.call_builtin('POINTS_EQUAL', F.col(END_COL), F.call_builtin('TO_GEOGRAPHY', curr_point)) == False)

nearest_neighbor_path = session.create_dataframe([[path, total_distance, 'naive']], schema=["PATH", "DISTANCE", "METHOD"])
nearest_neighbor_path.write.mode("overwrite").save_as_table("route")

